# Accuracy Benchmark

In [1]:
from pathlib import Path
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.patches import Patch
from matplotlib.transforms import Bbox
from utils.plotting_defaults import (
    autolabel_bars,
    delete_tex_cache,
    parse_logs_special,
    parse_logs_wide,
    plot_legend,
    set_style,
)

from fainder.utils import configure_run, load_input

configure_run("WARNING")
set_style()
Path("plots/accuracy_benchmark").mkdir(parents=True, exist_ok=True)

In [2]:
delete_tex_cache()

In [ ]:
def parse_logs(
    path: str, has_id: bool = True
) -> tuple[list[dict[str, Any]], list[dict[str, Any]]]:
    metrics = ["precision", "recall", "f1", "pruning_factor"]
    accuracy = []
    size_time = []
    cid = 1000
    for logfile in Path(path).iterdir():
        logs = load_input(logfile)
        config = logfile.stem.split("-")

        for mode, mode_data in [
            ("recall", logs["recall_mode_metrics"]),
            ("precision", logs["precision_mode_metrics"]),
        ]:
            for i, values in enumerate(mode_data):
                accuracy.extend(
                    [
                        {
                            "dataset": config[0],
                            "index_type": config[1],
                            "clustering_id": int(config[2]) if has_id else cid,
                            "algorithm": config[2 + has_id],
                            "n_clusters": int(config[3 + has_id][1:]),
                            "n_bins": int(config[4 + has_id][1:]),
                            "transform": config[5 + has_id],
                            "alpha": int(config[6 + has_id][1:]),
                            "query_category": config[7 + has_id],
                            "index_mode": mode,
                            "metric": metrics[i],
                            "value": value,
                        }
                        for value in values
                    ]
                )

        if config[7 + has_id] == "low_selectivity":
            # The index is the same for all query categories
            size_time.append(
                {
                    "dataset": config[0],
                    "index_type": config[1],
                    "clustering_id": int(config[2]) if has_id else cid,
                    "algorithm": config[2 + has_id],
                    "n_clusters": int(config[3 + has_id][1:]),
                    "n_bins": int(config[4 + has_id][1:]),
                    "transform": config[5 + has_id],
                    "alpha": int(config[6 + has_id][1:]),
                    "index_size": logs["index_size"] / 1024**2,
                    "runtime": (logs["recall_mode_time"] + logs["precision_mode_time"]) / 2,
                }
            )

        if not has_id:
            cid += 1

    return accuracy, size_time

In [4]:
grid_search = parse_logs("../logs/accuracy_benchmark/grid_search/")
k_cluster = parse_logs("../logs/accuracy_benchmark/k_cluster/", has_id=False)

accuracy = pd.DataFrame(grid_search[0] + k_cluster[0])
size_time = pd.DataFrame(grid_search[1] + k_cluster[1])

## Grid Search

#### SportsTables

In [5]:
# Precision of Fainder with full recall
accuracy[
    (accuracy["dataset"] == "sportstables")
    & (accuracy["index_type"] == "conversion")
    & (accuracy["index_mode"] == "recall")
    & (accuracy["metric"] == "precision")
    & (accuracy["algorithm"] == "kmeans")
    & (accuracy["n_bins"] == 5000)
].groupby(
    [
        "n_clusters",
        "transform",
        "alpha",
    ]
).agg({"value": ["mean", "std"]}).sort_values(("value", "mean"), ascending=False).head(30)

value          
                                mean       std
n_clusters transform alpha                    
4          bone      0      0.992848  0.033585
                     1      0.992809  0.033639
                     5      0.989430  0.053237
8          standard  5      0.970344  0.129085
10         standard  1      0.968082  0.091951
8          standard  0      0.966090  0.142406
                     1      0.966085  0.142367
110        standard  1      0.958228  0.128144
60         standard  1      0.951785  0.130393
230        standard  1      0.951262  0.115442
190        standard  1      0.950395  0.136830
150        standard  1      0.949773  0.143339
220        standard  1      0.945231  0.154635
180        quantile  1      0.938804  0.153726
160        quantile  1      0.938790  0.153280
110        quantile  1      0.938738  0.151160
190        quantile  1      0.938656  0.153789
130        quantile  1      0.938449  0.153510
170        quantile  1      0.938362  0.154102
50         standard  1      0.938068  0.181907
200        quantile  1      0.938016  0.155038
120        quantile  1      0.937533  0.153623
130        standard  1      0.937485  0.186498
210        quantile  1      0.937478  0.154719
250        standard  1      0.936380  0.152063
220        quantile  1      0.936290  0.158493
400        quantile  1      0.936282  0.149442
230        quantile  1      0.936260  0.158627
40         standard  1      0.935385  0.194487
150        quantile  1      0.935321  0.152998

In [6]:
# F1 of Fainder without guarantees
accuracy[
    (accuracy["dataset"] == "sportstables")
    & (accuracy["index_type"] == "rebinning")
    & (accuracy["index_mode"] == "recall")
    & (accuracy["metric"] == "f1")
    & (accuracy["algorithm"] == "kmeans")
    & (accuracy["n_bins"] == 5000)
].groupby(
    [
        "n_clusters",
        "transform",
        "alpha",
    ]
).agg({"value": ["mean", "std"]}).sort_values(("value", "mean"), ascending=False).head(30)

value          
                                mean       std
n_clusters transform alpha                    
10         standard  1      0.987843  0.035246
130        quantile  1      0.985482  0.027114
8          standard  5      0.985036  0.049230
                     0      0.984900  0.050931
                     1      0.984856  0.051138
120        quantile  1      0.984856  0.028139
110        quantile  1      0.984848  0.025930
60         standard  1      0.983516  0.053467
4          bone      0      0.982953  0.051045
                     5      0.982936  0.051315
                     1      0.982921  0.051299
100        quantile  1      0.982421  0.024361
80         quantile  1      0.982348  0.024707
90         quantile  1      0.982281  0.023810
70         quantile  1      0.982063  0.024637
                     0      0.981857  0.025153
                     5      0.981515  0.025004
60         quantile  1      0.980507  0.024778
240        standard  1      0.979660  0.069059
190        standard  1      0.978737  0.069301
230        standard  1      0.977952  0.064209
110        standard  1      0.977221  0.079555
250        standard  1      0.976466  0.073400
180        standard  1      0.975510  0.076714
120        standard  1      0.974234  0.079424
150        standard  1      0.973700  0.094609
200        standard  1      0.972852  0.080525
210        standard  1      0.972689  0.076500
140        quantile  1      0.969929  0.086898
           standard  1      0.968959  0.096737

In [7]:
# Index size of conversion indices
# NOTE: Rebinning indices have the same scaling but are 50% smaller
size_time[
    (size_time["dataset"] == "sportstables")
    & (size_time["index_type"] == "conversion")
    & (size_time["n_bins"] == 5000)
].groupby(
    [
        "algorithm",
        "n_clusters",
        "transform",
        "alpha",
    ]
).agg({"index_size": ["mean"], "runtime": ["mean"]}).sort_values(
    ("index_size", "mean"), ascending=True
).head(20)

index_size   runtime
                                           mean      mean
algorithm n_clusters transform alpha                     
kmeans    1000       standard  1       2.350555  0.208315
          950        standard  1       2.396408  0.189346
          900        standard  1       2.445648  0.185614
          1000       quantile  1       2.451035  0.119248
          950        quantile  1       2.646042  0.181789
          850        standard  1       2.666992  0.175365
          800        standard  1       2.690430  0.165283
          900        quantile  1       2.717819  0.170429
          850        quantile  1       2.853729  0.172809
          750        standard  1       2.874664  0.149767
          800        quantile  1       2.995377  0.156503
          700        standard  1       3.067627  0.155295
          750        quantile  1       3.213791  0.150051
          650        standard  1       3.254852  0.156424
          700        quantile  1       3.331345  0.144092
          600        standard  1       3.551727  0.132177
          650        quantile  1       3.563095  0.136941
          550        standard  1       3.775635  0.139762
          600        quantile  1       3.855255  0.123607
          550        quantile  1       4.150314  0.119942

#### Open Data

In [8]:
# Precision of Fainder with full recall
accuracy[
    (accuracy["dataset"] == "open_data_usa")
    & (accuracy["index_type"] == "conversion")
    & (accuracy["index_mode"] == "recall")
    & (accuracy["metric"] == "precision")
    & (accuracy["algorithm"] == "kmeans")
    & (accuracy["n_bins"] == 50000)
].groupby(
    [
        "n_clusters",
        "transform",
        "alpha",
    ]
).agg({"value": ["mean", "std"]}).sort_values(("value", "mean"), ascending=False).head(30)

value          
                                mean       std
n_clusters transform alpha                    
700        quantile  1      0.963599  0.046669
250        quantile  1      0.960875  0.053120
300        quantile  1      0.960672  0.053271
240        quantile  1      0.960220  0.054811
220        quantile  1      0.959459  0.055644
230        quantile  1      0.959225  0.056898
550        quantile  1      0.958212  0.054781
160        quantile  1      0.958196  0.057965
170        quantile  1      0.957957  0.057350
350        quantile  1      0.957788  0.056725
190        quantile  1      0.957595  0.057741
400        quantile  1      0.957355  0.057730
650        quantile  1      0.957350  0.055156
180        quantile  1      0.957158  0.059151
800        quantile  1      0.956387  0.055112
130        quantile  1      0.956055  0.060905
140        quantile  1      0.956046  0.060899
450        quantile  1      0.955388  0.059113
210        quantile  1      0.954995  0.062083
600        quantile  1      0.954968  0.058661
950        quantile  1      0.954841  0.056659
850        quantile  1      0.954733  0.057715
200        quantile  1      0.954728  0.063133
80         quantile  1      0.954610  0.064829
900        quantile  1      0.954276  0.057312
150        quantile  1      0.953961  0.063991
500        quantile  1      0.952493  0.062946
750        quantile  1      0.950755  0.063028
120        quantile  1      0.950206  0.069273
1000       quantile  1      0.949284  0.063408

In [9]:
# F1 of Fainder without guarantees
accuracy[
    (accuracy["dataset"] == "open_data_usa")
    & (accuracy["index_type"] == "rebinning")
    & (accuracy["index_mode"] == "recall")
    & (accuracy["metric"] == "f1")
    & (accuracy["algorithm"] == "kmeans")
    & (accuracy["n_bins"] == 50000)
].groupby(
    [
        "n_clusters",
        "transform",
        "alpha",
    ]
).agg({"value": ["mean", "std"]}).sort_values(("value", "mean"), ascending=False).head(30)

value          
                                mean       std
n_clusters transform alpha                    
160        quantile  1      0.976898  0.024355
850        quantile  1      0.976014  0.026904
230        quantile  1      0.975932  0.025535
240        quantile  1      0.975265  0.026499
300        quantile  1      0.975026  0.026732
170        quantile  1      0.974963  0.027550
250        quantile  1      0.974942  0.026810
1000       quantile  1      0.974841  0.028338
190        quantile  1      0.974780  0.027597
180        quantile  1      0.974368  0.028221
400        quantile  1      0.974256  0.028782
500        quantile  1      0.973957  0.029456
200        quantile  1      0.973927  0.029100
350        quantile  1      0.973885  0.028110
220        quantile  1      0.973812  0.028351
210        quantile  1      0.973787  0.028594
450        quantile  1      0.972968  0.030507
120        quantile  1      0.972721  0.030737
950        quantile  1      0.972552  0.032140
140        quantile  1      0.972113  0.031119
130        quantile  1      0.972112  0.031120
150        quantile  1      0.972079  0.031424
750        quantile  1      0.970945  0.033977
80         quantile  1      0.970556  0.034154
650        quantile  1      0.970364  0.034699
137        quantile  1      0.970293  0.034659
                     0      0.970210  0.034692
550        quantile  1      0.970178  0.035196
600        quantile  1      0.969224  0.036411
137        quantile  5      0.967898  0.038390

In [10]:
# Index size of conversion indices
# NOTE: Rebinning indices have the same scaling but are 50% smaller
size_time[
    (size_time["dataset"] == "open_data_usa")
    & (size_time["index_type"] == "conversion")
    & (size_time["n_bins"] == 50000)
].groupby(
    [
        "algorithm",
        "n_clusters",
        "transform",
        "alpha",
    ]
).agg({"index_size": ["mean"], "runtime": ["mean"]}).sort_values(
    ("index_size", "mean"), ascending=True
).head(30)

index_size     runtime
                                            mean        mean
algorithm n_clusters transform alpha                        
kmeans    1000       quantile  1      100.816238    0.133316
          950        quantile  1      104.729462    0.129662
          900        quantile  1      110.831268    0.125166
          850        quantile  1      116.721771    0.076559
          800        quantile  1      121.248062    0.114768
          750        quantile  1      130.607239    0.112924
          700        quantile  1      144.108170    0.106826
          650        quantile  1      145.652725    0.100544
          600        quantile  1      157.936386    0.100709
          550        quantile  1      171.697601    0.093058
          500        quantile  1      185.843292    0.056329
          450        quantile  1      205.151245    0.079572
          400        quantile  1      237.662369    0.080562
          350        quantile  1      265.173843    0.071917
          300        quantile  1      308.584534    0.066367
          250        quantile  1      345.501572  156.427362
          240        quantile  1      351.651581  152.107062
          230        quantile  1      360.495071  153.147588
          220        quantile  1      374.857239  150.604729
          210        quantile  1      395.279099  151.723865
          200        quantile  1      414.569626  153.421336
          190        quantile  1      444.070999  152.850036
          180        quantile  1      458.649445  153.593860
          170        quantile  1      481.749069  152.418798
          160        quantile  1      522.190445  154.560652
          150        quantile  1      531.565445  151.822898
          140        quantile  1      540.712326  154.831579
          137        quantile  5      550.013870  153.216204
                               1      551.372177  154.023907
                               0      551.617874  155.421631

#### GitTables

In [11]:
# Precision of Fainder with full recall
accuracy[
    (accuracy["dataset"] == "gittables")
    & (accuracy["index_type"] == "conversion")
    & (accuracy["index_mode"] == "recall")
    & (accuracy["metric"] == "precision")
    & (accuracy["algorithm"] == "kmeans")
    & (accuracy["n_bins"] == 100000)
].groupby(
    [
        "n_clusters",
        "transform",
        "alpha",
    ]
).agg({"value": ["mean", "std"]}).sort_values(("value", "mean"), ascending=False).head(30)

value          
                                mean       std
n_clusters transform alpha                    
750        quantile  1      0.932933  0.088259
1000       quantile  1      0.919060  0.108781
550        quantile  1      0.915222  0.113840
600        quantile  1      0.914681  0.115061
950        quantile  1      0.908623  0.123608
650        quantile  1      0.905817  0.127064
850        quantile  1      0.904152  0.130526
900        quantile  1      0.902845  0.131613
400        quantile  1      0.900231  0.134976
350        quantile  1      0.899845  0.137067
700        quantile  1      0.897367  0.139858
800        quantile  1      0.897083  0.141240
500        quantile  1      0.897047  0.140253
450        quantile  1      0.888853  0.152723
230        quantile  1      0.881325  0.164931
240        quantile  1      0.880376  0.161912
250        quantile  1      0.879489  0.163866
220        quantile  1      0.879341  0.166875
300        quantile  1      0.878224  0.168007
200        quantile  1      0.848775  0.207054
210        quantile  1      0.848683  0.207007
180        quantile  1      0.843739  0.213547
190        quantile  1      0.843492  0.213432
170        quantile  1      0.843464  0.213976
160        quantile  1      0.842629  0.215187
140        quantile  1      0.842528  0.213939
150        quantile  1      0.833424  0.226184
130        quantile  1      0.801656  0.270208
120        quantile  1      0.800601  0.269671
100        quantile  1      0.794426  0.278840

In [12]:
# F1 of Fainder without guarantees
accuracy[
    (accuracy["dataset"] == "gittables")
    & (accuracy["index_type"] == "rebinning")
    & (accuracy["index_mode"] == "recall")
    & (accuracy["metric"] == "f1")
    & (accuracy["algorithm"] == "kmeans")
    & (accuracy["n_bins"] == 100000)
].groupby(
    [
        "n_clusters",
        "transform",
        "alpha",
    ]
).agg({"value": ["mean", "std"]}).sort_values(("value", "mean"), ascending=False).head(30)

value          
                                mean       std
n_clusters transform alpha                    
750        quantile  1      0.962760  0.049449
1000       quantile  1      0.956283  0.058775
550        quantile  1      0.951792  0.065325
600        quantile  1      0.951599  0.065701
950        quantile  1      0.948634  0.069832
650        quantile  1      0.945523  0.074139
900        quantile  1      0.945097  0.074822
850        quantile  1      0.944172  0.076598
400        quantile  1      0.941779  0.079706
350        quantile  1      0.941192  0.081172
700        quantile  1      0.940157  0.082307
800        quantile  1      0.939657  0.083675
500        quantile  1      0.939562  0.083300
450        quantile  1      0.933534  0.092352
230        quantile  1      0.927898  0.101746
240        quantile  1      0.927861  0.098758
250        quantile  1      0.927050  0.100307
220        quantile  1      0.926467  0.103547
300        quantile  1      0.925637  0.103611
200        quantile  1      0.902926  0.134872
210        quantile  1      0.902901  0.134867
180        quantile  1      0.898924  0.139957
190        quantile  1      0.898739  0.139883
170        quantile  1      0.898711  0.140333
140        quantile  1      0.898298  0.140238
160        quantile  1      0.897949  0.141280
150        quantile  1      0.890190  0.151380
130        quantile  1      0.861112  0.191882
120        quantile  1      0.860633  0.191692
100        quantile  1      0.854005  0.201306

In [13]:
# Index size of conversion indices
# NOTE: Rebinning indices have the same scaling but are 50% smaller
size_time[
    (size_time["dataset"] == "gittables")
    & (size_time["index_type"] == "conversion")
    & (size_time["n_bins"] == 100000)
].groupby(
    [
        "algorithm",
        "n_clusters",
        "transform",
        "alpha",
    ]
).agg({"index_size": ["mean"], "runtime": ["mean"]}).sort_values(
    ("index_size", "mean"), ascending=True
).head(30)

index_size     runtime
                                              mean        mean
algorithm n_clusters transform alpha                          
kmeans    1000       quantile  1      16093.904129    0.372123
          950        quantile  1      16762.399170    0.410569
          900        quantile  1      17814.311935    0.451463
          850        quantile  1      18359.495773    0.398447
          800        quantile  1      18994.175674    0.423860
          750        quantile  1      19814.889755    0.337874
          700        quantile  1      20414.964218    0.424860
          650        quantile  1      22264.083893    0.381489
          600        quantile  1      23740.201324    0.349818
          550        quantile  1      25317.365570    0.363786
          500        quantile  1      26870.686127    0.426665
          450        quantile  1      30164.245972    0.428479
          400        quantile  1      31024.653381    0.410388
          350        quantile  1      34936.833664    0.410811
          300        quantile  1      40042.690720    0.420866
          250        quantile  1      47449.313538  189.745158
          240        quantile  1      49860.239502  191.355690
          230        quantile  1      52139.410690  192.458867
          220        quantile  1      53712.163254  196.041628
          210        quantile  1      54750.361511  196.648372
          200        quantile  1      57099.163345  202.731891
          190        quantile  1      59380.832336  203.259755
          180        quantile  1      61542.257187  204.827117
          170        quantile  1      63947.755661  206.003986
          160        quantile  1      68595.173599  210.381612
          150        quantile  1      74019.019470  211.119597
          140        quantile  1      82109.190018  220.858424
          130        quantile  1      89430.673050  229.712366
          113        quantile  5      91298.392487  235.083404
                               0      91299.542496  225.994972

## Baseline Comparison

In [14]:
datasets = ["sportstables", "open_data_usa", "gittables"]
queries = ["low_selectivity", "mid_selectivity", "high_selectivity"]

In [15]:
baseline_list = []
for dataset in datasets:
    for query_set in queries:
        for approach, metric_name in [
            ("pscan", "hist"),
            ("ndist", "dist"),
            ("binsort", "binsort"),
        ]:
            acc_logs = load_input(
                f"../logs/accuracy_benchmark/baseline_comp/{dataset}-{approach}-{query_set}.zst"
            )
            perf_logs = parse_logs_wide(
                f"../logs/accuracy_benchmark/baseline_comp/{dataset}-{approach}-{query_set}.log"
            )
            baseline_list.append(
                [
                    dataset,
                    approach,
                    query_set,
                    perf_logs["query_collection_time"],
                    acc_logs[f"{metric_name}_time"],
                    np.mean(acc_logs[f"{metric_name}_metrics"][0]),
                    np.mean(acc_logs[f"{metric_name}_metrics"][1]),
                    np.mean(acc_logs[f"{metric_name}_metrics"][2]),
                    np.mean(acc_logs[f"{metric_name}_metrics"][3]),
                ]
            )

        # Fainder rebinning and conversion
        for index_mode in ["rebinning", "conversion"]:
            acc_logs = load_input(
                f"../logs/accuracy_benchmark/baseline_comp/{dataset}-{index_mode}-{query_set}.zst"
            )
            runtimes = parse_logs_special(
                f"../logs/accuracy_benchmark/baseline_comp/{dataset}-{index_mode}-{query_set}.log"
            )
            baseline_list.append(
                [
                    dataset,
                    f"{index_mode}-precision",
                    query_set,
                    runtimes[0],
                    acc_logs["precision_mode_time"],
                    np.mean(acc_logs["precision_mode_metrics"][0]),
                    np.mean(acc_logs["precision_mode_metrics"][1]),
                    np.mean(acc_logs["precision_mode_metrics"][2]),
                    np.mean(acc_logs["precision_mode_metrics"][3]),
                ]
            )
            baseline_list.append(
                [
                    dataset,
                    f"{index_mode}-recall",
                    query_set,
                    runtimes[1],
                    acc_logs["recall_mode_time"],
                    np.mean(acc_logs["recall_mode_metrics"][0]),
                    np.mean(acc_logs["recall_mode_metrics"][1]),
                    np.mean(acc_logs["recall_mode_metrics"][2]),
                    np.mean(acc_logs["recall_mode_metrics"][3]),
                ]
            )

        # Fainder exact
        acc_logs = load_input(
            f"../logs/accuracy_benchmark/baseline_comp/{dataset}-exact-{query_set}.zst"
        )
        baseline_list.append(
            [
                dataset,
                "exact",
                query_set,
                acc_logs["precision_time"] + acc_logs["recall_time"] + acc_logs["iterative_time"],
                acc_logs["precision_time"] + acc_logs["recall_time"] + acc_logs["iterative_time"],
                1,  # Metrics not logged because approach is exact
                1,
                1,
                None,
            ]
        )

baseline_comp = pd.DataFrame(
    baseline_list,
    columns=[
        "dataset",
        "approach",
        "queries",
        "precise_time",
        "total_time",
        "precision",
        "recall",
        "f1",
        "pruning_factor",
    ],
)

In [16]:
baseline_comp.head()

,dataset,approach,queries,precise_time,total_time,precision,recall,f1,pruning_factor
0,sportstables,pscan,low_selectivity,54.878682,54.878811,1.000000,1.000000,1.000000,0.064941
1,sportstables,ndist,low_selectivity,45.583732,46.701013,0.978830,0.926949,0.937479,0.064375
2,sportstables,binsort,low_selectivity,3.009388,3.009566,1.000000,0.998957,0.999457,-1.000000
3,sportstables,rebinning-precision,low_selectivity,0.298398,0.298644,1.000000,0.879412,0.909875,0.060713
4,sportstables,rebinning-recall,low_selectivity,0.311775,0.312036,0.919609,0.980042,0.938961,0.067635


In [17]:
def plot_baseline_barchart(dataset: str, metric: str, height: float) -> None:
    width = 0.5
    fig, ax = plt.subplots(1, 1, figsize=(2.4, height))

    ax.bar(
        x=0,
        height=baseline_comp[
            (baseline_comp["dataset"] == dataset) & (baseline_comp["approach"] == "pscan")
        ][metric].mean()
        * 100,
        width=width,
        color=sns.color_palette()[0],
        edgecolor="black",
        clip_on=False,
    )

    for i, approach in enumerate(["ndist", "rebinning-recall", "conversion-recall"]):
        for j, query_set in enumerate(["low_selectivity", "mid_selectivity", "high_selectivity"]):
            ax.bar(
                x=1.5 * width + i * 3.8 * width + j * 1.1 * width,
                height=baseline_comp[
                    (baseline_comp["dataset"] == dataset)
                    & (baseline_comp["approach"] == approach)
                    & (baseline_comp["queries"] == query_set)
                ][metric].mean()
                * 100,
                width=width,
                color=sns.color_palette()[j + 1],
                edgecolor="black",
                clip_on=False,
            )

    if metric == "pruning_factor":
        # For technical reasons, the pruning factor is not logged for the exact approach
        # It is, however, guaranteed to be the same as for pscan as the method is exact
        exact_height = baseline_comp[
            (baseline_comp["dataset"] == dataset) & (baseline_comp["approach"] == "pscan")
        ][metric].mean()
    else:
        exact_height = baseline_comp[
            (baseline_comp["dataset"] == dataset) & (baseline_comp["approach"] == "exact")
        ][metric].mean()
    ax.bar(
        x=12.9 * width,
        height=exact_height * 100,
        width=width,
        color=sns.color_palette()[0],
        edgecolor="black",
        clip_on=False,
    )

    ax.set_xticks(
        [0, 2.6 * width, 6.4 * width, 10.2 * width, 12.9 * width],
        [
            r"\texttt{profile-}" "\n" r"\texttt{scan}",
            r"\texttt{normal-}" "\n" r"\texttt{dist}",
            r"\textsc{F. Approx}" "\n" r"low mem.",
            r"\textsc{F. Approx}" "\n" r"full rec.",
            r"\textsc{Fainder}" "\n" r"\textsc{Exact}",
        ],
    )
    ax.set_xlim(-0.6 * width, 13.5 * width)
    ax.set_ylim(0, 100)
    sns.despine()
    autolabel_bars(ax, precision=3, decimal_precision=1)

    bbox = fig.get_tightbbox()

    plt.tight_layout(pad=1.02)
    plt.savefig(
        f"plots/accuracy_benchmark/bar_{metric}_{dataset}.pdf",
        bbox_inches="tight",
        pad_inches=0.01,
    )

    ax.set_ylabel(r"$F_1$ score (\%)")
    bbox = fig.get_tightbbox()
    label_bbox = Bbox(((bbox.x0, bbox.y0), (0.07, bbox.y1)))
    plt.savefig(f"plots/accuracy_benchmark/bar_{metric}_ylabel.pdf", bbox_inches=label_bbox)
    plt.close()

In [18]:
height = 1.1
plot_baseline_barchart("sportstables", "f1", height)
plot_baseline_barchart("open_data_usa", "f1", height)
plot_baseline_barchart("gittables", "f1", height)

plot_legend(
    "plots/accuracy_benchmark/bar_legend.pdf",
    handles=[
        Patch(facecolor=sns.color_palette()[0], edgecolor="black", label="All queries"),
        Patch(facecolor=sns.color_palette()[1], edgecolor="black", label="Low selectivity"),
        Patch(facecolor=sns.color_palette()[2], edgecolor="black", label="Mid selectivity"),
        Patch(facecolor=sns.color_palette()[3], edgecolor="black", label="High selectivity"),
    ],
    labels=["All queries", "Low selectivity", "Mid selectivity", "High selectivity"],
    ncol=4,
)

In [19]:
def plot_approximate_barchart(dataset: str, metric: str, ylabel: str, precision: int = 3) -> None:
    width = 0.5
    fig, ax = plt.subplots(1, 1, figsize=(1.95, 1.15))

    for i, approach in enumerate(["ndist", "rebinning-recall", "conversion-recall"]):
        for j, query_set in enumerate(["low_selectivity", "mid_selectivity", "high_selectivity"]):
            ax.bar(
                x=i * 3.8 * width + j * 1.1 * width,
                height=baseline_comp[
                    (baseline_comp["dataset"] == dataset)
                    & (baseline_comp["approach"] == approach)
                    & (baseline_comp["queries"] == query_set)
                ][metric].mean()
                * 100,
                width=width,
                color=sns.color_palette()[j + 1],
                edgecolor="black",
            )

    ax.set_xticks(
        [1.1 * width, 4.9 * width, 8.7 * width],
        [
            r"\texttt{normal-}" "\n" r"\texttt{dist}",
            r"\textsc{F. Approx}" "\n" r"low mem.",
            r"\textsc{F. Approx}" "\n" r"full rec.",
        ],
    )
    ax.set_xlim(-0.6 * width, 10.4 * width)
    ax.set_ylabel(ylabel)
    ax.set_ylim(0, 100)
    sns.despine()
    autolabel_bars(ax, precision=precision, decimal_precision=2)

    plt.tight_layout(pad=1.02)
    plt.savefig(
        f"plots/accuracy_benchmark/bar_approx_{metric}_{dataset}.pdf",
        bbox_inches="tight",
        pad_inches=0.01,
    )
    plt.close()

In [20]:
plot_approximate_barchart("gittables", "precision", r"Precision (\%)", precision=2)
plot_approximate_barchart("gittables", "pruning_factor", r"Pruning factor (\%)")

plot_legend(
    "plots/accuracy_benchmark/bar_approx_legend.pdf",
    handles=[
        Patch(facecolor=sns.color_palette()[1], edgecolor="black", label="Low selectivity"),
        Patch(facecolor=sns.color_palette()[2], edgecolor="black", label="Mid selectivity"),
        Patch(facecolor=sns.color_palette()[3], edgecolor="black", label="High selectivity"),
    ],
    labels=["Low selectivity", "Mid selectivity", "High selectivity"],
    ncol=3,
)

In [21]:
def plot_baseline_scatter(
    dataset: str, metric: str, ylabel: str, queries: str, height: float
) -> None:
    fig, ax = plt.subplots(1, 1, figsize=(1.23, height))

    if queries == "all":
        data = baseline_comp[
            (baseline_comp["dataset"] == dataset)
            & ~baseline_comp["approach"].isin(["conversion-precision", "rebinning-precision"])
        ]
    else:
        data = baseline_comp[
            (baseline_comp["dataset"] == dataset)
            & (baseline_comp["queries"] == queries)
            & ~baseline_comp["approach"].isin(["conversion-precision", "rebinning-precision"])
        ]
    data = (
        data.groupby(["approach"])
        .agg({"precise_time": "mean", metric: "mean"})
        .reindex(["pscan", "ndist", "binsort", "exact", "rebinning-recall", "conversion-recall"])
    )

    ax.scatter(
        data["precise_time"],
        data[metric] * 100,
        c=sns.color_palette()[:6],
        clip_on=False,
    )

    ax.grid(True, which="major", axis="y", linestyle="--", linewidth=0.5, alpha=0.3, color="gray")
    ax.set_xlabel("Time (s)")
    ax.set_xscale("log")
    ax.set_ylim(0, 100)
    sns.despine()

    bbox = fig.get_tightbbox()

    plt.tight_layout(pad=1.02)
    plt.savefig(
        f"plots/accuracy_benchmark/scatter_{metric}_{dataset}_{queries}.pdf",
        bbox_inches="tight",
        pad_inches=0.01,
    )

    ax.set_ylabel(ylabel)
    bbox = fig.get_tightbbox()
    label_bbox = Bbox(((bbox.x0, bbox.y0), (0.07, bbox.y1)))
    plt.savefig(f"plots/accuracy_benchmark/scatter_{metric}_ylabel.pdf", bbox_inches=label_bbox)
    plt.close()

In [22]:
height = 1.1
plot_baseline_scatter("sportstables", "f1", r"$F_1$ score (\%)", "all", height)
plot_baseline_scatter("open_data_usa", "f1", r"$F_1$ score (\%)", "all", height)
plot_baseline_scatter("gittables", "f1", r"$F_1$ score (\%)", "all", height)

plot_legend(
    "plots/accuracy_benchmark/scatter_legend.pdf",
    handles=[Patch(facecolor=sns.color_palette()[i], edgecolor="black") for i in range(6)],
    labels=[
        r"\pscan{}",
        r"\ndist{}",
        r"\binsort{}",
        r"\exact{}",
        r"\approximate{} low mem.",
        r"\approximate{} full rec.",
    ],
    ncol=3,
)

## LLM Workload

In [23]:
llm_workload_list = []
for dataset in ["sportstables", "open_data_usa", "gittables"]:
    for approach, metric_name in [("ndist", "dist")]:
        acc_logs = load_input(f"../logs/accuracy_benchmark/llm/{dataset}-{approach}.zst")
        perf_logs = parse_logs_wide(f"../logs/accuracy_benchmark/llm/{dataset}-{approach}.log")
        llm_workload_list.append(
            [
                dataset,
                approach,
                perf_logs["query_collection_time"],
                acc_logs[f"{metric_name}_time"],
                np.mean(acc_logs[f"{metric_name}_metrics"][0]),
                np.mean(acc_logs[f"{metric_name}_metrics"][1]),
                np.mean(acc_logs[f"{metric_name}_metrics"][2]),
                np.mean(acc_logs[f"{metric_name}_metrics"][3]),
            ]
        )

    # Fainder rebinning and conversion
    for index_mode in ["rebinning", "conversion"]:
        acc_logs = load_input(f"../logs/accuracy_benchmark/llm/{dataset}-{index_mode}.zst")
        runtimes = parse_logs_special(f"../logs/accuracy_benchmark/llm/{dataset}-{index_mode}.log")
        llm_workload_list.append(
            [
                dataset,
                f"{index_mode}-precision",
                runtimes[0],
                acc_logs["precision_mode_time"],
                np.mean(acc_logs["precision_mode_metrics"][0]),
                np.mean(acc_logs["precision_mode_metrics"][1]),
                np.mean(acc_logs["precision_mode_metrics"][2]),
                np.mean(acc_logs["precision_mode_metrics"][3]),
            ]
        )
        llm_workload_list.append(
            [
                dataset,
                f"{index_mode}-recall",
                runtimes[1],
                acc_logs["recall_mode_time"],
                np.mean(acc_logs["recall_mode_metrics"][0]),
                np.mean(acc_logs["recall_mode_metrics"][1]),
                np.mean(acc_logs["recall_mode_metrics"][2]),
                np.mean(acc_logs["recall_mode_metrics"][3]),
            ]
        )

llm_workload = pd.DataFrame(
    llm_workload_list,
    columns=[
        "dataset",
        "approach",
        "precise_time",
        "total_time",
        "precision",
        "recall",
        "f1",
        "pruning_factor",
    ],
)

In [24]:
llm_workload

,dataset,approach,precise_time,total_time,precision,recall,f1,pruning_factor
0,sportstables,ndist,67.224687,68.341222,0.968147,0.949730,0.949052,0.544303
1,sportstables,rebinning-precision,1.307101,1.307373,1.000000,0.905956,0.944611,0.530038
2,sportstables,rebinning-recall,1.685054,1.685313,0.959596,0.988332,0.968360,0.566059
3,sportstables,conversion-precision,1.328422,1.328680,1.000000,0.882629,0.929658,0.520594
4,sportstables,conversion-recall,1.303118,1.303293,0.940290,0.999849,0.959607,0.575459
5,open_data_usa,ndist,176.906457,177.924347,0.738804,0.612974,0.649006,0.515364
6,open_data_usa,rebinning-precision,3.358881,3.359142,1.000000,0.784510,0.866643,0.510127
7,open_data_usa,rebinning-recall,3.375606,3.375771,0.989864,0.882612,0.929943,0.554564
8,open_data_usa,conversion-precision,1.839633,1.839809,1.000000,0.654058,0.773053,0.439275
9,open_data_usa,conversion-recall,3.045192,3.045367,0.983415,0.999937,0.991453,0.629861


In [25]:
def plot_llm_barchart(dataset: str, metric: str, ylabel: str, precision: int = 3) -> None:
    width = 0.5
    fig, ax = plt.subplots(1, 1, figsize=(1.23, 1.15))

    for i, approach in enumerate(["ndist", "rebinning-recall", "conversion-recall"]):
        ax.bar(
            x=i * 1.5 * width,
            height=llm_workload[
                (llm_workload["dataset"] == dataset) & (llm_workload["approach"] == approach)
            ][metric].mean()
            * 100,
            width=width,
            color=sns.color_palette()[i],
            edgecolor="black",
        )

    ax.set_xticks([])
    ax.set_xlim(-0.6 * width, 3.6 * width)
    ax.set_ylim(0, 100)
    sns.despine()
    autolabel_bars(ax, precision=precision, decimal_precision=2)

    plt.tight_layout(pad=1.02)
    plt.savefig(
        f"plots/accuracy_benchmark/barchart_llm_{metric}_{dataset}.pdf",
        bbox_inches="tight",
        pad_inches=0.01,
    )

    ax.set_ylabel(ylabel)
    bbox = fig.get_tightbbox()
    label_bbox = Bbox(((bbox.x0, bbox.y0), (0.07, bbox.y1)))
    plt.savefig("plots/accuracy_benchmark/barchart_llm_ylabel.pdf", bbox_inches=label_bbox)
    plt.close()

In [26]:
plot_llm_barchart("sportstables", "f1", r"$F_1$ score (\%)", precision=3)
plot_llm_barchart("open_data_usa", "f1", r"$F_1$ score (\%)", precision=3)
plot_llm_barchart("gittables", "f1", r"$F_1$ score (\%)", precision=3)

plot_legend(
    "plots/accuracy_benchmark/barchart_llm_legend.pdf",
    handles=[
        Patch(facecolor=sns.color_palette()[0], edgecolor="black"),
        Patch(facecolor=sns.color_palette()[1], edgecolor="black"),
        Patch(facecolor=sns.color_palette()[2], edgecolor="black"),
    ],
    labels=[
        r"\texttt{normal-dist}",
        r"\approximate{} low mem.",
        r"\approximate{} full rec.",
    ],
    ncol=3,
)